# Tarea Visualización Avanzada

Para la realización de esta tarea se ha utilizado el dataset *"Spotify Dataset 1921-2020, 600k+ Tracks"* de Kaggle.

Para acceder a la web será desde [aquí](https://www.kaggle.com/datasets/yamaerenay/spotify-dataset-19212020-600k-tracks/code).

## Ficheros

- artists.csv: contiene métricas de popularidad de cada artista.
- tracks.csv: contiene los detalles de cada canción.
- dict_artists.csv (creado): contiene un listado de artistas recomendados para fans de cada artista, ordenados de mejor a peor. Originalmente estaba en formato JSON y no tenía la columna de orden.
- dict_track_artist.csv (creado): contiene un listado de canciones y el artista que la creó. Pueden ser varios artistas para cada canción. 

## Comentarios

Se han excluido las canciones anteriores a 1980 de modo que se reduzcan los tamaños de los ficheros de forma considerable. Además, se han eliminado los artistas sin canciones.

In [8]:
# !pip install kagglehub pandas numpy matplotlib seaborn plotly

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
init_notebook_mode(connected=True)

In [3]:
# Descarga de los datos

# if not os.path.exists("./data/"):
#     os.makedirs("./data", exist_ok=True)
# if not (os.path.exists("./data/artists.csv") and os.path.exists("./data/tracks.csv") and os.path.exists("./data/dict_artists.json")):
#     path = kagglehub.dataset_download("yamaerenay/spotify-dataset-19212020-600k-tracks")
#     shutil.move(path, "./data/", copy_function=shutil.copy)
#     for file in os.listdir("./data/1"):
#         shutil.move(os.path.join("./data/1", file), "./data/")
#     shutil.rmtree(path)

In [49]:
artists_df = pd.read_csv("./data/artists.csv", encoding="utf-8")
tracks_df = pd.read_csv("./data/tracks.csv", encoding="utf-8")
dict_artists_df = pd.read_csv("./data/dict_artists.csv", encoding="utf-8")
dict_track_artist_df = pd.read_csv("./data/dict_track_artist.csv", encoding="utf-8")

In [5]:
# Creación del diccionario de artistas y sus artistas recomendados
# with open('./data/dict_artists.json', 'r', encoding='utf-8') as f:
#     data = json.load(f)
# dict_df = pd.DataFrame(list(data.items()), columns=['artist', 'recommended_artists']).explode('recommended_artists')
# dict_df['order'] = dict_df.groupby('artist').cumcount() + 1 # Añade la columna de orden
# dict_df.to_csv("./data/dict_artists.csv", index=False, encoding="utf-8")

In [6]:
# Creación del diccionario de canciones y sus artistas
# dict_track_artist = tracks_df[['id', 'id_artists', 'artists']]
# for col in ['id_artists', 'artists']:
#     dict_track_artist[col] = dict_track_artist[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# dict_track_artist = dict_track_artist.explode(['id_artists', 'artists']).reset_index(drop=True)
# dict_track_artist = dict_track_artist.rename(columns={
#     'id_artists': 'id_artist',
#     'artists': 'artist'
# })
# dict_track_artist.to_csv("./data/dict_track_artist.csv", index=False, encoding="utf-8")

In [12]:
# ver de tracks_df las filas cuya release_date tenga una longitud de 4 caracteres
tracks_df[tracks_df['release_date'].str.len() == 4]
# añadir a estos casos un "-01-01" al final de la fecha
tracks_df.loc[tracks_df['release_date'].str.len() == 4, 'release_date'] += '-01-01'
# convertir la columna release_date a datetime
tracks_df['release_date'] = pd.to_datetime(tracks_df['release_date'], errors='coerce')

In [ ]:
# Cantidad de canciones por año de lanzamiento
tracks_df['release_date'].value_counts()

release_date
2020    13937
1999    12479
1998    12478
1997    12347
1996    12055
2019    11907
1995    11584
2014    11252
2016    11083
2018    10936
1994    10709
2015    10591
2013    10557
1990    10181
1992    10041
1991     9908
2017     9889
2012     9744
1989     9735
2008     9714
2011     9689
2010     9597
2009     9593
2004     9251
1988     8902
1986     8881
2002     8877
2005     8856
2007     8770
2006     8751
1987     8343
1980     8279
2003     8208
1985     7868
1984     7777
1981     7599
2000     7557
1982     7516
2001     7252
1983     7105
1993     7032
2021     6281
Name: count, dtype: int64

In [ ]:
# Vamos a graficarlo

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed